# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy_database.csv")

# Display sample data
city_data_df

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nelson Bay,AU,-32.7167,152.1500,70.00,92,100,4.00,moderate rain
1,1,Vaini,TO,-21.2000,-175.2000,74.26,80,71,22.03,light rain
2,2,Port Elizabeth,ZA,-33.9180,25.5701,57.49,94,75,2.30,broken clouds
3,3,Pochutla,MX,15.7432,-96.4661,76.82,92,100,3.74,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.14,68,4,16.73,clear sky
...,...,...,...,...,...,...,...,...,...,...
676,676,Ukiah,US,39.1502,-123.2078,72.64,50,0,1.01,clear sky
677,677,Kasempa,ZM,-13.4584,25.8338,71.31,47,82,9.24,broken clouds
678,678,Williston,US,48.1470,-103.6180,56.32,51,100,7.00,overcast clouds
679,679,Molina,CL,-35.1167,-71.2833,56.93,72,93,5.75,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))


# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nelson Bay,AU,-32.7167,152.1500,70.00,92,100,4.00,moderate rain
1,1,Vaini,TO,-21.2000,-175.2000,74.26,80,71,22.03,light rain
3,3,Pochutla,MX,15.7432,-96.4661,76.82,92,100,3.74,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.14,68,4,16.73,clear sky
9,9,Victoria,HK,22.2855,114.1577,86.97,63,35,5.99,scattered clouds
10,10,Saint-Philippe,RE,-21.3585,55.7679,74.79,78,49,9.31,light rain
14,14,Dong Hoi,VN,17.4833,106.6000,73.65,84,99,14.85,overcast clouds
16,16,Rikitea,PF,-23.1203,-134.9692,71.10,70,99,19.44,overcast clouds
17,17,Hilo,US,19.7297,-155.0900,80.33,80,40,11.50,scattered clouds
20,20,Carnarvon,AU,-24.8667,113.6333,71.74,55,1,11.95,clear sky


In [5]:
preferred_cities_df.count()

City                   274
City.1                 274
Country                271
Lat                    274
Lng                    274
Max Temp               274
Humidity               274
Cloudiness             274
Wind Speed             274
Current Description    274
dtype: int64

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
preferred_cities_df.isnull().sum()

City                   0
City.1                 0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City,City.1,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nelson Bay,AU,-32.7167,152.1500,70.00,92,100,4.00,moderate rain
1,1,Vaini,TO,-21.2000,-175.2000,74.26,80,71,22.03,light rain
3,3,Pochutla,MX,15.7432,-96.4661,76.82,92,100,3.74,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.14,68,4,16.73,clear sky
9,9,Victoria,HK,22.2855,114.1577,86.97,63,35,5.99,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
668,668,Carlsbad,US,33.1581,-117.3506,74.62,88,75,3.44,mist
672,672,Santa Cruz,BO,-17.8000,-63.1667,72.86,83,75,6.91,thunderstorm with light rain
674,674,Namatanai,PG,-3.6667,152.4333,85.30,65,99,3.44,light rain
676,676,Ukiah,US,39.1502,-123.2078,72.64,50,0,1.01,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [8]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City.1", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head(10)

,City.1,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nelson Bay,AU,70.00,moderate rain,-32.7167,152.1500,
1,Vaini,TO,74.26,light rain,-21.2000,-175.2000,
3,Pochutla,MX,76.82,overcast clouds,15.7432,-96.4661,
4,Atuona,PF,77.14,clear sky,-9.8000,-139.0333,
9,Victoria,HK,86.97,scattered clouds,22.2855,114.1577,
10,Saint-Philippe,RE,74.79,light rain,-21.3585,55.7679,
14,Dong Hoi,VN,73.65,overcast clouds,17.4833,106.6000,
16,Rikitea,PF,71.10,overcast clouds,-23.1203,-134.9692,
17,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,
20,Carnarvon,AU,71.74,clear sky,-24.8667,113.6333,


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [9]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df = preferred_cities_df[["City.1", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()

# Display sample data
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df



,City.1,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nelson Bay,AU,70.00,moderate rain,-32.7167,152.1500,
1,Vaini,TO,74.26,light rain,-21.2000,-175.2000,
3,Pochutla,MX,76.82,overcast clouds,15.7432,-96.4661,
4,Atuona,PF,77.14,clear sky,-9.8000,-139.0333,
9,Victoria,HK,86.97,scattered clouds,22.2855,114.1577,
...,...,...,...,...,...,...,...
668,Carlsbad,US,74.62,mist,33.1581,-117.3506,
672,Santa Cruz,BO,72.86,thunderstorm with light rain,-17.8000,-63.1667,
674,Namatanai,PG,85.30,light rain,-3.6667,152.4333,
676,Ukiah,US,72.64,clear sky,39.1502,-123.2078,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [10]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [11]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# Display sample data
hotel_df

,City.1,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nelson Bay,AU,70.00,moderate rain,-32.7167,152.1500,Mantra Nelson Bay
1,Vaini,TO,74.26,light rain,-21.2000,-175.2000,Keleti Beach Resort
3,Pochutla,MX,76.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
4,Atuona,PF,77.14,clear sky,-9.8000,-139.0333,Villa Enata
9,Victoria,HK,86.97,scattered clouds,22.2855,114.1577,Mini Central
...,...,...,...,...,...,...,...
668,Carlsbad,US,74.62,mist,33.1581,-117.3506,Carlsbad Inn Beach Resort
672,Santa Cruz,BO,72.86,thunderstorm with light rain,-17.8000,-63.1667,Aviador Hotel Boutique
674,Namatanai,PG,85.30,light rain,-3.6667,152.4333,Seagulls Inn Namatani
676,Ukiah,US,72.64,clear sky,39.1502,-123.2078,Hampton Inn Ukiah


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [13]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']

# Display sample data
clean_hotel_df

,City.1,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nelson Bay,AU,70.00,moderate rain,-32.7167,152.1500,Mantra Nelson Bay
1,Vaini,TO,74.26,light rain,-21.2000,-175.2000,Keleti Beach Resort
3,Pochutla,MX,76.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
4,Atuona,PF,77.14,clear sky,-9.8000,-139.0333,Villa Enata
9,Victoria,HK,86.97,scattered clouds,22.2855,114.1577,Mini Central
...,...,...,...,...,...,...,...
668,Carlsbad,US,74.62,mist,33.1581,-117.3506,Carlsbad Inn Beach Resort
672,Santa Cruz,BO,72.86,thunderstorm with light rain,-17.8000,-63.1667,Aviador Hotel Boutique
674,Namatanai,PG,85.30,light rain,-3.6667,152.4333,Seagulls Inn Namatani
676,Ukiah,US,72.64,clear sky,39.1502,-123.2078,Hampton Inn Ukiah


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [14]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [15]:
# Set the file name.
output_data_file = ("WeatherPy_Vacation.csv")

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [24]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City.1</dt><dd>{City.1}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [25]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

KeyError: 'City'

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [ ]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [ ]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Create a figure to add the Google map as a layer
fig.add_layer(heat_layer)

# Display the figure containing the map
fig.add_layer(marker_layer)

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig